# NAVER API로 핀테크 뉴스 제목 수집

## 1. requests 모듈 설치
* [설명서 링크 바로가기 >](https://requests.readthedocs.io/en/latest/)
* 파이썬 라이브러리
* 인터넷(웹 서버, API)에 접속해서 데이터를 가져오는 역할을 함.
* requests.get(url)

In [1]:
!pip install requests

## 2. requests 사용법
```python
import requests

url = "접속주소url" # url
payload = dict(key=value) # parameter
headers = dict(key=value) # headers
r = requests.get(url.params=payload) # 서버에 url + parameter로 요청보내기 # r은 관용적으로 사용하는 것
print(r.url) # 서버에 요청을 보낸 url 출력
print(r.status_code) #서버에서 보내준 응답코드 200 정상, 400, 500은 오류
response = r.text # r.content, r.json()
```

* r.text: utf-8로 인코딩해서 보여줌 한글이 잘 보임
* r.content:서버가 보내준 그대로의 자료     => 나중에 HTML의 자료를 뽑을 때 사용
* r.json():requests 모듈 내부의 json모듈로 text를 json으로 자동 변환

## 3. requests로 네이버 API에서 자료 수집
* **install**
    * pandas
    * python-dotenv

In [6]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.7 MB/s  0:00:00 38.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 37.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]━━━━ 1/2 [pandas]


In [8]:
!pip install python-dotenv

In [35]:
import os
import sys
sys.path.append("../../") # 컴퓨터 내 myfunc 파일을 불러오기 위해 추가 / 작업 파일과 env 파일의 위치가 달라 경로 지정 후 실행

import requests
import json
import pandas as pd
from myfunc import text_clean # 수집 데이터 정리
import time # API 수집 속도 조절

from dotenv import load_dotenv
load_dotenv("../../env")  # 가장 아래에 두면 true가 출력되기 때문에 좋다. / 작업 파일과 env 파일의 위치가 달라 경로 지정 후 실행

True

## 4. NAVER API 파라미터 설정 후 최대로 수집

* **파라미터	타입	필수 여부	설명**
    * query:	String	Y	검색어. UTF-8로 인코딩되어야 합니다.  
    * display:	Integer	N	한 번에 표시할 검색 결과 개수(기본값: 10, 최댓값: 100)  
    * start:	Integer	N	검색 시작 위치(기본값: 1, 최댓값: 1000)  
    * sort:	String	N	검색 결과 정렬 방법  
        - sim: 정확도순으로 내림차순 정렬(기본값)  
        - date: 날짜순으로 내림차순 정렬  

# 5. 결과

### 5-1. keyword를 입력받아서 blog, news, book 카테고리에서 데이터 수집 후 결과를 각각 csv 파일로 저장하기
* naver_api에서 키워드와 관련된 news, blog, book 자료를 1100개씩 검색하고
* keyword_news_result.csv, keyword_blog_result.csv, keyword_book_result.csv 형태로 저장되도록 하세요.

In [47]:
# 키워드 입력
keyword = input('검색할 키워드를 입력하세요: ')

# url에 붙일 키워드를 category로 순서대로 넣는다.
for category in ['blog', 'news', 'book']:
    
    result = {}
    start_num = 1
    while True:
            # NAVER 검색 API의 검색 시작 위치는 최대 1000까지만 가능. 이 조건을 피하기 위한 코드
        if start_num == 1001:
            start_num -= 1
        elif start_num > 1000:
            break

        # start_num 출력
        print("start_num: ", start_num)

        url = f"https://openapi.naver.com/v1/search/{category}"
        payload = dict(query=keyword, display=100,start=start_num,sort='sim')
        headers = {"X-Naver-Client-Id" : os.getenv('Client_Id'),
                   "X-Naver-Client-Secret":os.getenv('Client_Secret')}
        r = requests.get(url, params=payload, headers=headers)
        data = r.json()

        # API 응답 상태 확인(알 수 없는 에러가 계속 발생해서 추가했습니다.)
        if r.status_code !=200:
            print(f"[Error] 호출 실패! 상태 코드: {r.status_code}")
            print("에러 메시지:", data)
            break

        if 'items' not in data:
            print(f"[Info]결과 없음:{data}")
            break

        # 데이터 정리 및 전처리
        for item in data['items']:
            for key,value in item.items():
                if key in ('title', 'description'):
                    value = text_clean(value)
                result.setdefault(key, []).append(value)
                
        # 다음 검색 시작 위치 추가
        start_num += 100
    
    # dataframe 생성  
    df = pd.DataFrame(result)

    # 폴더 생성
    dir_name = "data"
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        print(f"{dir_name}을 생성했습니다.")
    else:
        print(f"{dir_name}이 이미 있습니다.")

    # 파일 저장
    df.to_csv(f'./data/{keyword}_{category}_result.csv',encoding="utf-8-sig")

    # API 호출 속도 제한
    time.sleep(0.5)

검색할 키워드를 입력하세요:  핀테크


start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data을 생성했습니다.
start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이 이미 있습니다.
start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이 이미 있습니다.


### 5-2. naver_api 수집 코드 함수화

In [49]:
# 함수 생성(고정인수) => 인자를 한 개만 받을 수 있다. 
def naver_search(keyword):
    
    # url에 붙일 키워드를 category로 순서대로 넣는다.
    for category in ['blog', 'news', 'book']:

        result = {}
        start_num = 1
        while True:
            # NAVER 검색 API의 검색 시작 위치는 최대 1000까지만 가능. 이 조건을 피하기 위한 코드
            if start_num == 1001:
                start_num -= 1
            elif start_num > 1000:
                break

            # 검색 시작 위치 출력
            print("start_num: ", start_num)     

            url = f"https://openapi.naver.com/v1/search/{category}"
            payload = dict(query=keyword, display=100,start=start_num,sort='sim')
            headers = {"X-Naver-Client-Id" : os.getenv('Client_Id'),
                       "X-Naver-Client-Secret":os.getenv('Client_Secret')}
            r = requests.get(url, params=payload, headers=headers)
            data = r.json()

            # API 응답 상태 확인(알 수 없는 에러가 계속 발생해서 추가했습니다.)
            if r.status_code !=200:
                print(f"[Error] 호출 실패!!! 상태 코드: {r.status_code}")
                print("에러 메시지:", data)
                break
    
            if 'items' not in data:
                print(f"[Info]결과 없음:{data}")
                break
                
            # 데이터 정리 및 전처리
            for item in data['items']:
                for key,value in item.items():
                    if key in ('title', 'description'):
                        value = text_clean(value)
                    result.setdefault(key, []).append(value)

            # 다음 검색 시작 위치 추가
            start_num += 100

        # dataframe 생성
        df = pd.DataFrame(result)

        # 폴더 생성
        dir_name = "data"
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
            print(f"{dir_name}을(를) 생성했습니다.")
        else:
            print(f"{dir_name}이(가) 이미 있습니다.")

        # 파일 생성
        df.to_csv(f"./data/{keyword}_{category}_result_function_PosArgs.csv", encoding="utf-8-sig")

        # API 호출 속도 제한 
        time.sleep(0.5)

* 'naver_search' 함수 실행

In [50]:
naver_search("핀테크")

start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이(가) 이미 있습니다.
start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이(가) 이미 있습니다.
start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이(가) 이미 있습니다.


### 5-3. 키워드와 카테고리를 입력하는대로 검색하고 출력하게 하기

In [51]:
# 함수 생성(가변인수) => 인자를 모든 인자를 받을 수 있다.
def naver_search(keyword, *categories):
    
    # url에 붙일 키워드를 category로 순서대로 넣는다.
    for category in categories:

        result = {}
        start_num = 1
        while True:
            # NAVER 검색 API의 검색 시작 위치는 최대 1000까지만 가능. 이 조건을 피하기 위한 코드
            if start_num == 1001:
                start_num -= 1
            elif start_num > 1000:
                break
                
            # 검색 시작 위치 출력
            print("start_num: ", start_num)

            url = f"https://openapi.naver.com/v1/search/{category}"
            payload = dict(query=keyword, display=100,start=start_num,sort='sim')
            headers = {"X-Naver-Client-Id" : os.getenv('Client_Id'),
                       "X-Naver-Client-Secret":os.getenv('Client_Secret')}
            r = requests.get(url, params=payload, headers=headers)
            data = r.json()

            # API 응답 상태 확인(알 수 없는 에러가 계속 발생해서 추가했습니다.)
            if r.status_code !=200:
                print(f"[Error] 호출 실패!!! 상태 코드: {r.status_code}")
                print("에러 메시지:", data)
                break
    
            if 'items' not in data:
                print(f"[Info]결과 없음:{data}")
                break

            # 데이터 정리 및 전처리
            for item in data['items']:
                for key,value in item.items():
                    if key in ('title', 'description'):
                        value = text_clean(value)
                    result.setdefault(key, []).append(value)

            # 다음 검색 시작 위치 추가
            start_num += 100
        
        # dataframe 생성
        df = pd.DataFrame(result)

        # 폴더 생성
        dir_name = "data"
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
            print(f"{dir_name}을(를) 생성했습니다.")
        else:
            print(f"{dir_name}이(가) 이미 있습니다.")

        # 파일 생성
        df.to_csv(f"./data/{keyword}_{category}_result_function_Args.csv")
        
        # API 호출 속도 제한 
        time.sleep(0.5)

* 'naver_search' 함수 실행

In [52]:
naver_search("핀테크", 'news', 'blog', 'book')

start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이(가) 이미 있습니다.
start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이(가) 이미 있습니다.
start_num:  1
start_num:  101
start_num:  201
start_num:  301
start_num:  401
start_num:  501
start_num:  601
start_num:  701
start_num:  801
start_num:  901
start_num:  1000
data이(가) 이미 있습니다.
